# library loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.eXTiGDB5Ne/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.AKfINq9l47/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.RVnyQuuikq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://deb.debian.or

In [ ]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 정규표현식
import pandas as pd #csv저장
from datetime import datetime # 시간 체크

# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 화면 주소 정의
## 스킨케어
a = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001'
## 마스크팩
b = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010009&gateCd=Drawer'
## 클렌징
c = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010010&gateCd=Drawer'
## 선케어
d = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010011&gateCd=Drawer'

# driver setting

In [ ]:
#  driver setting
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions() 
ops.add_experimental_option('excludeSwitches', ['enable-logging'])
ops.add_argument('--headless')        # Head-less 설정
ops.add_argument('--no-sandbox')
ops.add_argument('--disable-dev-shm-usage')
# ops.add_argument('__headlees') ## 확인해보자

# chromedriver path
driver = webdriver.Chrome('chromedriver', options=ops)

## practice

In [ ]:
#driver.find_element(By.CLASS_NAME, 'prd_brand').text

'피지오겔'

In [ ]:
# a = driver.find_element(By.CLASS_NAME, 'price').text
# # a =a.split(sep='\n')
# a

'38,000\n원\n21,510\n원\n혜택 정보'

In [ ]:
#driver.get(url='https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000154540&dispCatNo=100000100010008&trackingCd=Cat100000100010008_Small')

In [ ]:
# driver.find_element(By.CLASS_NAME, 'price').text

'7,900\n원'

## defined function

In [ ]:
## 리뷰개수를 반환해주는 함수

def rv_num2():

    # 리뷰 수 //*[@id="gdasContentsArea"]/div/div[1]/div/div[2]/p[1]/em
  global q # 총 리뷰수
  global pn # 리뷰 페이지수
  global pc # 마지막 페이지의 리뷰수
  q = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text[1:-1]
  q = int(re.sub(r"[^0-9]", "", q))
  if q == 0:  # 리뷰수가 없는 경우
      pn = 'zero'
      pc = 'zero'
  elif 0 < q <= 10:  # 리뷰수가 10이하인 경우
      pn = 'one'
      pc = q
  else: # 리뷰수가 11이상
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
      pn = (q // 10) + 1 
      pc = q % 10

def rv_info2():
  try :
      global pname   
      pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
      global tstar
      # #gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong
      tstar = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong').text
  except:
      print('별점은없어')  # 리뷰가 없는경우 tstar를 가져오지 못함.(리뷰가 없음)
      pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text

# 브랜드명과 가격정보
def rv_brand():
  try:
    global brand
    brand = driver.find_element(By.CLASS_NAME, 'prd_brand').text
    global price
    price = driver.find_element(By.CLASS_NAME, 'price').text
  except:
    brand = '-'
    price = '-'

In [ ]:
## 리뷰의 키워드를 반환해주는 함수

def find_kw(i):
    try:  # //*[@id="gdasList"]/li[10]/div[2]/div[2]    # //*[@id="gdasList"]/li[10]/div[2]/div[3]
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/div[2]').text
    except:
        x = 'no keyword' # 키워드 선택을 안하는 경우
    finally:
        kw_l.append(x)

## 리뷰를 가져오는 함수

def find_text(i):   # //*[@id="gdasList"]/li[1]/div[2]/div[2]/p   헤드 없고 키워드 없고 내용만  
        # 키워드 없는 헤드 //*[@id="gdasList"]/li[6]/div[2]/p    ## //*[@id="gdasList"]/li[6]/div[2]/div[2]   # 내용인데 헤드가 있으면
    try:
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/div[3]').text
        # txt inner
        rv = x.split(sep='\n')
        if len(rv) == 1: # 리뷰가 한줄로 작성된 경우
            text = rv[0]
        else:
            text = rv[0]+ rv[-1] # 리뷰 앞문장과 마지막 문장 가져오기
        # print(text)
    except:
        try:  # 리뷰 아우터만 존재하는 경우
            text = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/p').text
        except:
            text = 'no text'  # 해당 리뷰 구조가 없으면 리뷰가 없는걸로
    finally:
        rv_l.append(text)



In [ ]:
## 한페이지 안에 가져올 리뷰정보를 반환하는 함수

def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        time.sleep(0.1)
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/div[1]/span[1]/span').text
        x = list(x)
        star = int(x[-2])
        star_l.append(star)
        find_kw(i)   # keywords function
        find_text(i) # 리뷰 함수
        brand_l.append(brand)
        pname_l.append(pname)
        price_l.append(price)
        tstar_l.append(tstar)
        # print('ok')
## 한 상품에서 가져올 정보를 반환하는 함수

def one_product2(pn, pc):
     # 가장 첫페이지 리뷰가져오기
    list_up()
    print('1_page')
    for l in range(2, pn+1): # 리뷰 페이지 수 만큼 
        if str(l)[-1] == '0':
            try:  # 10, 20 등 뒤에 다음 버튼이 오는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 페이지 넘검
                moment
                list_up() # 리뷰내용 가져오는 함수(default = 10)
                print(l,'_page')
                driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click() # 다음 페이지로
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                if len(a) == 10: # 리뷰수 10개
                    list_up()
                else: 
                    list_up(pc) # pc랑 a가 같은 수임
                print(l,'_page')
            except: # 클릭후 다음 버튼이 없는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 마지막 페이지
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                list_up(len(a))  # pc가 0일 수도 있어서
                print(l, '_page')
                
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'_page')
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            moment
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
                print(l,'_page^')
            else:
                list_up()
                print(l,'_page')
        elif str(l)[-1] == '1': 
            print(l) # 11,21 등의 페이지는 버튼클릭이 다음 버튼이라 넣을 내용이 없음.
        elif l == (pn): # 현재가 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            if pc != 0:
                list_up(pc)
                print(l,'_page')
            else:
                list_up() # 0으로 끝나면 10이니
                print(l,'_page')
            # 끝자리수만큼 포문 pc list_up(pc)
            # print(l)
        elif '2' <= str(l)[-1] <= '9' and l != pn: # 0,1 마지막 페이지가 아닌경우 다음버튼 누른후 10개씩 가져옴
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'_page')

In [ ]:
## 상품의 리뷰를 가져오는 함수
### 위 함수들의 진행하는데 있어 생각하지 못한 오류 들이 발생할 수 있어서 예외처리하고자 함.

def try_pro():
  try:
      rv_num2()
      rv_info2()
      rv_brand()  # 새로 추가됨(브랜드, 가격)
      if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
          non_l.append(pname) # 상품명 리스트에는 안올라감
      elif pn == 'one':
          # 1~10사이
        list_up(pc) # 10개라도 여기선 pc가 q임 ㅎ
        print(pname, '_수집')
      else:
        one_product2(pn, pc)
        print(pname, '_수집')
  except:
        # pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
      fail_l.append(pname)
      print("??? 문제 발생 ???", pname)

## action

In [ ]:
# 리스트 정의
pname_l = []
brand_l = []
price_l = []
tstar_l = []
star_l = []
kw_l = []
rv_l = []

non_l = []
fail_l = []

In [ ]:
# page moving
driver.get(url=a)
driver.implicitly_wait(5)
# 기본페이지 이동
# 스킨케어_토너 페이지로 이동
driver.find_element(By.XPATH, f'//*[@id="Contents"]/div/div[1]/ul/li[1]/a').click()
driver.implicitly_wait(5)
# 리뷰 48개로 선택
driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
driver.implicitly_wait(5)

# 리뷰 컨테이너 요소를 만나기 위한 작업(explicitly wait지정)
driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{2}]/li[{1}]/div/a/img').click()
driver.implicitly_wait(5)
driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
## 리뷰컨테이너를 만날때까지 최대 6초 기다림
moment = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont'))) # 해당요소를 만나야만 가능

# 첫페이지 화면으로
driver.back()

In [ ]:
# crawler 
# 48개의 상품들의 정보를 10페이지 수집
for x in range(2,11):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                driver.implicitly_wait(5)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                moment
                try_pro()
                driver.back()
                driver.implicitly_wait(5)
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        moment
    except:
        continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
23 _page
??? 문제 발생 ??? 바이오더마 센시비오 토너 250ml
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page
10 _page
11
12 _page
13 _page
14 _page
15 _page
16 _page
17 _page
18 _page
19 _page
20 _page
20 _page
21
22 _page
23 _page
??? 문제 발생 ??? 라운드랩 1025 독도 토너+로션기획 (토너 200ml+로션 200ml)
1_page
??? 문제 발생 ??? 닥터지 레드 블레미쉬 포 맨 올인원 크림 150ml 기획세트(+30ml 증정)
1_page
2 _page
3 _page
4 _page
??? 문제 발생 ??? 닥터지 더모이스처 배리어 D 로션세트(200+30)
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page
10 _page
11
12 _page
13 _page
14 _page
??? 문제 발생 ??? 라네즈옴므 크림 스킨 옴므 올인원 150ml 기획(+25ml+25ml 증정)
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page
10 _page
11
12 _page
13 _page
14 _page
15 _page
16 _page
17 _page
18 _page
19 _page
20 _page
20 _page
21
22 _page
23 _page
24 _page
25 _page
26 _page
27 _page
28 _page
29 _page
30 _page
30 _page
31
32 _page
33 _page
34 _page
35 _page
36 _page
37 _page
38 _page
39 _pag

In [ ]:
# 다음 버튼을 누르는 액션
driver.find_element(By.PARTIAL_LINK_TEXT, '다음').click()
driver.implicitly_wait(5)

In [ ]:
# crawler 
# 48개의 상품 6페이지를 수집
for x in range(2,7):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                driver.implicitly_wait(5)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                moment
                try_pro()
                driver.back()
                driver.implicitly_wait(5)
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        moment
    except:
        continue

1_page
2 _page
??? 문제 발생 ??? 라끄베르 옴므리차지 2종 기획
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page
10 _page
11
??? 문제 발생 ??? [김지훈 Pick] 미프 비타맥스 올인원 로션 리필 세트 기획 (본품 200ml+리필 150ml*2ea)
1_page
2 _page
??? 문제 발생 ??? 라끄베르 옴므리차지 올인원에센스 기획
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page
10 _page
11
12 _page
13 _page
14 _page
15 _page
16 _page
17 _page
18 _page
19 _page
20 _page
20 _page
21
22 _page
23 _page
24 _page
25 _page
26 _page
27 _page
28 _page
29 _page
30 _page
30 _page
31
32 _page
33 _page
34 _page
35 _page
36 _page
37 _page
38 _page
39 _page
40 _page
40 _page
41
42 _page
43 _page
44 _page
45 _page
46 _page
47 _page
48 _page
49 _page
50 _page
50 _page
51
52 _page
53 _page
54 _page
55 _page
56 _page
??? 문제 발생 ??? 식물나라 제주 탄산수 모공토너 패드 70매 (NEW)
1_page
2 _page
3 _page
미프 스킨로션 세트(스킨180ml+ 로션180ml) _수집
별점은없어
별점은없어
1_page
2 _page
질레트 랩스 모이스처라이저100ml _수집
1_page
2 _page
3 _page
4 _page
5 _page
6 _page
7 _page
8 _page
9 _page
10 _page

In [ ]:
driver.close()

In [ ]:
df = pd.DataFrame({'brand': brand_l, 'pname':pname_l,'price':price_l, 'tstar':tstar_l,'star':star_l,'keyword':kw_l, 'rv':rv_l})
df.to_csv('/content/drive/MyDrive/datasets.csv', sep = ',', encoding = 'utf-8-sig', index=False)

In [ ]:
print('끝난 시간: ',datetime.now())

끝난 시간:  2023-01-27 16:07:18.560029
